
### Scraping Soil Monitoring Data

This notebook is designed to scrape search engine results related to soil monitoring, reporting, and verification (MRV). 
It retrieves and processes relevant URLs and domains to analyze the most frequently mentioned sources.

#### Steps:
1. Install and import required libraries.
2. Define a web scraping function to collect search results.
3. Process and clean the extracted data.
4. Analyze the results (top domains, keyword presence, etc.).


In [1]:
# Import necessary libraries
import pandas as pd
import random
from datetime import datetime, date
import time
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Install required libraries
!pip install tldextract  # Extract top-level domains from URLs

import tldextract
from urllib.parse import urlparse

You should consider upgrading via the '/Users/lucilerabeau/code/LucileRb/scraping-soil-monitoring/.venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/lucilerabeau/code/LucileRb/scraping-soil-monitoring/.venv/bin/python3 -m pip install --upgrade pip' command.


In [12]:
print(webdriver.__version__)

4.28.1


In [ ]:
print(f"Selenium version : {webdriver.__version__}")

In [3]:
stop

NameError: name 'stop' is not defined

#### Function: `scraping_soil`

The `scraping_soil` function is designed to scrape Google search results for a given set of search terms.  
It collects and processes search results from multiple pages, extracting key information such as:  
- Titles  
- URLs  
- Search result text snippets  
- Page position  
- Type of content (SEO vs. SEA)  

**Usage:**  
Call the function with a list of search terms and the number of pages to scrape. The function returns a pandas DataFrame containing the extracted data.  

**Example:**  
```python
search_terms = ["soil monitoring", "carbon sequestration"]
df_results = scrapator(search_terms, 3)
print(df_results.head())

In [ ]:
def scraping_soil(search_terms, page_number):

    """
    Scrapes Google SERP (Search Engine Results Pages) for given search terms.

    Parameters:
    search_terms (list): List of queries to search for.
    page_number (int): Number of pages to scrape per search term.

    Returns:
    pd.DataFrame: A DataFrame containing extracted URLs, titles, and metadata.
    """

    start_time = datetime.now()
    print(f'Program started at: {start_time.strftime("%Y-%m-%d %H:%M:%S")}\n')

    df = pd.DataFrame()
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options = options)

    print('***** Scraping in progress *****')

    for term in search_terms:
        print(f'\n**** Searching for term: {term} ****\n')
        driver.get('https://google.com')

        # Accept cookies if prompted
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#L2AGLb.tHlp8d'))
            ).click()
        except:
            pass

        time.sleep(round(random.uniform(1, 10), 1))

        # Locate and use search bar
        search_box = driver.find_element(By.NAME, 'q')
        search_box.clear()
        search_box.send_keys(term)
        time.sleep(1)
        search_box.send_keys(Keys.RETURN)

        time.sleep(round(random.uniform(1, 10), 1))

        all_data = []  # Store extracted data

        for page in range(1, page_number + 1):
            print(f'*** Scraping page {page} ***')

            # Wait for page load / solve captcha if needed
            time.sleep(100)

            articles_SEO = driver.find_elements(By.CLASS_NAME, 'MjjYud')
            articles_SEA = driver.find_elements(By.CLASS_NAME, 'uEierd')

            # Extract SEO articles
            for position, article in enumerate(articles_SEO, start=1):
                try:
                    title = article.find_element(By.TAG_NAME, 'h3').text
                    link = article.find_element(By.TAG_NAME, 'a').get_attribute('href')
                    content = article.text
                    has_image = 'Yes' if article.find_elements(By.CLASS_NAME, 'Z26q7c.UK95Uc.Sth6v') else 'No'
                    all_data.append([link, title, content, term, page, position, 'SEO', has_image])
                except:
                    continue  # Skip if element not found

            # Extract SEA (paid ads) articles
            for position, ad in enumerate(articles_SEA, start=1):
                try:
                    title = ad.find_element(By.CLASS_NAME, 'CCgQ5.vCa9Yd.QfkTvb.N8QANc.MUxGbd.v0nnCb').text
                    link = ad.find_element(By.TAG_NAME, 'a').get_attribute('href')
                    content = ad.text
                    has_image = 'Yes' if ad.find_elements(By.CLASS_NAME, 'g-img.ZGomKf') else 'No'
                    all_data.append([link, title, content, term, page, position, 'SEA', has_image])
                except:
                    continue  # Skip if element not found

            # Navigate to the next page
            try:
                next_button = driver.find_element(By.XPATH, "//*[contains(text(),'Suivant')]")
                driver.execute_script("arguments[0].scrollIntoView();", next_button)
                next_button.click()
                time.sleep(round(random.uniform(1, 10), 1))
            except:
                print("No more pages available.")
                break

        # Convert to DataFrame
        df_term = pd.DataFrame(all_data, columns = ['url', 'title', 'content', 'search_term', 'page', 'position', 'type', 'image'])
        df_term.dropna(subset=['title'], inplace = True)
        df_term['company'] = df_term['url'].apply(lambda x: tldextract.extract(x).domain)
        df_term['domain'] = df_term['url'].apply(lambda x: urlparse(x).netloc)

        df = pd.concat([df, df_term], ignore_index = True)

    df['Date'] = date.today().strftime('%Y-%m-%d')

    driver.quit()
    end_time = datetime.now()
    print(f'Program completed at: {end_time.strftime("%Y-%m-%d %H:%M:%S")}')
    print(f'Duration: {end_time - start_time}')

    return df

In [ ]:
# List of search queries related to soil monitoring and verification
queries = [
    'Monitor* AND Report* AND Verif* AND MRV AND soil*',
    #soil monitoring reporting verification'
    ]

### Manual Intervention During Scraping  

This script uses Selenium to scrape Google search results, which involves opening a web browser.  
During execution, manual intervention may be required in the following cases:  

- **Google CAPTCHA:** If Google detects unusual activity, it may prompt a CAPTCHA challenge that needs to be solved manually.  
- **Consent pop-ups:** The script attempts to close the Google cookie consent pop-up, but in some cases, manual confirmation may be needed.  
- **Page navigation issues:** If the script gets stuck on a page, refreshing or clicking manually might be necessary.  

⚠ **Recommendation:** Run the script in a visible browser session and monitor its progress to handle any manual actions when required.  


In [ ]:
# Run scraping
df = scraping_soil(
    search_terms = queries,
    page_number = 10 # please select the number of pages you wish to scrape
    )

Program started at: 2025-06-10 10:35:31



***** Scraping in progress *****

**** Searching for term: Monitor* AND Report* AND Verif* AND MRV AND soil* ****

*** Scraping page 1 ***
*** Scraping page 2 ***
*** Scraping page 3 ***
*** Scraping page 4 ***
*** Scraping page 5 ***
*** Scraping page 6 ***
*** Scraping page 7 ***
*** Scraping page 8 ***
*** Scraping page 9 ***
*** Scraping page 10 ***
Program completed at: 2025-06-10 10:53:25
Duration: 0:17:54.697686


In [ ]:
# Remove empty rows from the DataFrame and reset index
df = df.loc[df['url'] != 'vide']
df = df.reset_index(drop = True)

In [ ]:
df['Country'] = 'France'

In [ ]:
df

,url,title,content,search_term,page,position,type,image,company,domain,Date,Country
0,https://www.horizon-europe.gouv.fr/monitoring-...,"Monitoring, reporting and verification of soil...","Monitoring, reporting and verification of soil...",Monitor* AND Report* AND Verif* AND MRV AND soil*,1,1,SEO,No,horizon-europe,www.horizon-europe.gouv.fr,2025-06-10,France
1,https://cordis.europa.eu/programme/id/HORIZON_...,"Monitoring, reporting and verification of soil...","Monitoring, reporting and verification of soil...",Monitor* AND Report* AND Verif* AND MRV AND soil*,1,3,SEO,No,europa,cordis.europa.eu,2025-06-10,France
2,https://unece.org/sustainable-energy/monitorin...,,"Autres questions\nWhat is mrv monitoring, repo...",Monitor* AND Report* AND Verif* AND MRV AND soil*,1,4,SEO,No,unece,unece.org,2025-06-10,France
3,https://4p1000.org/mrv-systems-related-to-soil...,MRV systems related to Soil health in the cont...,MRV systems related to Soil health in the cont...,Monitor* AND Report* AND Verif* AND MRV AND soil*,1,5,SEO,No,4p1000,4p1000.org,2025-06-10,France
4,https://www.sciencedirect.com/science/article/...,Solutions and insights for agricultural monito...,Solutions and insights for agricultural monito...,Monitor* AND Report* AND Verif* AND MRV AND soil*,1,6,SEO,No,sciencedirect,www.sciencedirect.com,2025-06-10,France
...,...,...,...,...,...,...,...,...,...,...,...,...
97,https://cgspace.cgiar.org/items/c29a7507-13a3-...,"Monitoring, reporting, and verification system...","Monitoring, reporting, and verification system...",Monitor* AND Report* AND Verif* AND MRV AND soil*,10,6,SEO,No,cgiar,cgspace.cgiar.org,2025-06-10,France
98,https://www.icos-cp.eu/news-and-events/news/fl...,"FLUXES, The European Greenhouse Gas Bulletin: MRV","FLUXES, The European Greenhouse Gas Bulletin: ...",Monitor* AND Report* AND Verif* AND MRV AND soil*,10,7,SEO,No,icos-cp,www.icos-cp.eu,2025-06-10,France
99,https://journals.plos.org/plosone/article?id=1...,"Moving from Measuring, Reporting, Verification...","Moving from Measuring, Reporting, Verification...",Monitor* AND Report* AND Verif* AND MRV AND soil*,10,8,SEO,No,plos,journals.plos.org,2025-06-10,France
100,https://www.naro.go.jp/publicity_report/public...,"Handbook of Monitoring, Reporting, and Verific...","Handbook of Monitoring, Reporting, and Verific...",Monitor* AND Report* AND Verif* AND MRV AND soil*,10,9,SEO,No,naro,www.naro.go.jp,2025-06-10,France


In [ ]:
# Display the column names of the DataFrame
df.columns

Index(['url', 'title', 'content', 'search_term', 'page', 'position', 'type',
       'image', 'company', 'domain', 'Date', 'Country'],
      dtype='object')

In [ ]:
# Show the 20 most frequently appearing domains in the dataset
df['domain'].value_counts()[:20]

domain
www.icos-cp.eu                      3
www.horizon-europe.gouv.fr          2
openknowledge.fao.org               2
www.researchgate.net                2
research.wur.nl                     2
www.academia.edu                    2
pubmed.ncbi.nlm.nih.gov             2
pureportal.ilvo.be                  2
irc-orcasa.eu                       2
hal.inrae.fr                        2
unece.org                           2
4p1000.org                          2
www.un-redd.org                     2
www.isric.org                       2
www.sciencedirect.com               2
www.deloitte.com                    2
co2re.org                           1
onlinelibrary.wiley.com             1
ieaghg.org                          1
assets.publishing.service.gov.uk    1
Name: count, dtype: int64

#### Exporting Results

After scraping and processing the search results, the data is saved as a CSV file.  
The filename includes the current date (`YYYY-MM-DD`) to keep track of different scraping sessions.  
This allows easy comparison of results across multiple executions.  

In [ ]:
# Export the results to a CSV file
# Please adapt the filename with the filepath of your choice.
df.to_csv(f'scraping_results_{date.today().strftime("%Y-%m-%d")}.csv', index = False)